In [8]:
import pandas as pd
import numpy as np

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string


from sklearn.preprocessing import LabelEncoder


df = pd.read_csv("imdb-dataset/IMDB Dataset.csv")

df.head()

#displaying all columns

print(df.columns)


#Effectuez des opérations de nettoyage des textes (suppression de la ponctuation, des stop words, mise en minuscules, etc.)

nltk.download('stopwords')
nltk.download('punkt')

stop_words = set(stopwords.words('english'))
df['cleaned_reviews'] = df['review'].apply(lambda x: ' '.join([word for word in word_tokenize(x.lower()) if word not in stop_words and word not in string.punctuation]))

print(df['cleaned_reviews'].head())



#encodage des labels pour transformer les labels catégoriels en valeurs numériques

le = LabelEncoder()
df['sentiment_encoded'] = le.fit_transform(df['sentiment'])







Index(['review', 'sentiment'], dtype='object')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ADEM\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ADEM\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
